In [1]:
######Creating the Markov Chain Model########

In [11]:
import pandas as pd

In [2]:
def get_row_total(row):
    """given a row, get sum of the values"""
    return sum(row.values())

In [3]:
def normalize_dict(dictionary):
    """given dictionary has values that are lists
        set the new value to a list where corresponding
        elements have been normalized"""
    
    for key, values_list in dictionary.items():
        total = sum(values_list)
        normed_values = []
        #find the normalized value and append
        for value in values_list:
            normed_value = value/total
            normed_values.append(normed_value)
        #set key equal to the new list of normed values
        dictionary[key] = normed_values
    return dictionary

In [4]:
def turn_to_dict_with_lists(nested_dict):
    """turn a nested dictionary into a dict with lists
        ex: {d1:{k1:v1, k2:v2, k3, v3}} --> {d1: [v1, v2, v3]}""" 
    new_dict = {}
    for row, columns in nested_dict.items():
        listed_values = make_arbitrary_list_from(columns)
        new_dict[row] = listed_values
    return new_dict

In [5]:
def make_arbitrary_list_from(dictionary):
    """creates a list where the first value from 
    keyvalue pair is index 0, and so forth"""
    new_list = []
    for value in dictionary.values():
        new_list.append(value)
    return new_list

In [6]:
def add_to_row_dict(row_dict, value_list):
    """given list of values, """
    for i in range(0, len(value_list) - 1): # - 1 because we don't want to go out of range
        current = value_list[i]
        following = value_list[i + 1]
        #initialize dict with chords
        if current not in row_dict:
            row_dict[current] = {'I':0, 'ii':0, 'iii':0, 'IV':0, 'V':0, 'vi':0, 'vii':0}
        current_row = row_dict[current]   
        if following not in row_dict[current]:
            current_row[following] = 1
        else:
            current_row[following] += 1
    return row_dict    
    

In [7]:
def create_markov_chain(dictionary):
    """Given a dictionary with containing lists 
        of values, count the odds of an element proceeding it"""
    row_dict = {}
    for value_list in dictionary.values():
        row_dict = add_to_row_dict(row_dict, value_list)
    
    #in order to make it easy for pandas, we turn it into 1 dictionary, with lists
    formatted_dict = turn_to_dict_with_lists(row_dict)
    normalized_dictionary = normalize_dict(formatted_dict)
    return normalized_dictionary
                             

In [8]:
queen_chords = {'Dont Stop Me':['I', 'ii', 'vi', 'IV', 'V', 'I'], 
                   'bohemian rhapsody opening': ['vi', 'I', 'I', 'ii', 'V', 'V', 'I'], 
                   'bohemian rhapsody breakdown':['vi', 'ii', 'vi', 'ii', 'ii', 'V', 'ii', 'V', 'I'],
                  'Fat Bottom Girls': ['I', 'V', 'vi', 'I', "vi", "IV", "I"],
                  'We are the champions' : ['I', 'iii', 'vi', 'IV', 'V', 'I']}

In [13]:
queens_table = create_markov_chain(queen_chords)

In [14]:
chord_df = pd.DataFrame.from_dict(queens_table, orient='index')

In [15]:
chord_df = chord_df.rename(columns={0: "I", 1: "ii", 2: "iii", 3: "IV", 4: "V", 5: "vi", 6: "vii"})

In [16]:
######Here begins generation functions##############################

import random
import numpy as np

In [17]:
def generate_chord(chord_row, rng):
    chords = chord_row.index.to_numpy()
    values = chord_row.values
    #generate the next chord with weighted randomization
    next_chord = np.random.choice(chords, 1, p=values)
    return next_chord


In [18]:
def get_next_chord(chord_df, chord):
    rng = random.uniform(0, 1)
    curr_chord_row = chord_df.loc[chord, :]
    next_chord = generate_chord(curr_chord_row, rng)
    return next_chord

In [19]:
def generate_chord_prog(first_chord, num_of_times, chord_df):
    #initialize list with the first chord
    generated_prog = [first_chord]
    current = first_chord
    for i in range(0, num_of_times - 1): 
        next_chord = get_next_chord(chord_df, current)[0]
        generated_prog.append(next_chord)
        #reassign current so that next iteration won't run on a loop
        current = next_chord
    return generated_prog
    

In [20]:
def generate_new_queen_song(chord_df):
    first_chord = 'I'
    return generate_chord_prog( 'I', 8, chord_df)
        

In [21]:
chord_df

,I,ii,iii,IV,V,vi,vii
I,0.166667,0.333333,0.166667,0.000000,0.166667,0.166667,0.0
ii,0.000000,0.166667,0.000000,0.000000,0.500000,0.333333,0.0
vi,0.285714,0.285714,0.000000,0.428571,0.000000,0.000000,0.0
IV,0.333333,0.000000,0.000000,0.000000,0.666667,0.000000,0.0
V,0.571429,0.142857,0.000000,0.000000,0.142857,0.142857,0.0
iii,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0


In [25]:
generate_new_queen_song(chord_df)

['I', 'iii', 'vi', 'ii', 'vi', 'IV', 'V', 'ii']